# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


- Nama: Rayhan Egar Sadtya Nugraha
- NIM: 225150201111014

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
# ! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print(data_latih.shape[0])
print(data_uji.shape[0])

120
30


In [6]:
data_uji.columns

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [7]:
label_uji = data_uji.pop('variety')

In [8]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
114,5.8,2.8,5.1,2.4
139,6.9,3.1,5.4,2.1
88,5.6,3.0,4.1,1.3
106,4.9,2.5,4.5,1.7
73,6.1,2.8,4.7,1.2


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [88]:
from collections import Counter

def hitung_prior(list_kelas):
    n_data = len(list_kelas)
    prior = Counter(list_kelas)
    for key in prior.keys():
        prior[key] = prior[key]/n_data
    return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [89]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Setosa': 0.36666666666666664, 'Versicolor': 0.325, 'Virginica': 0.30833333333333335})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [11]:
def hitung_rata2_std_kelas(input_data):
    list_columns = input_data.columns[:-1]
    class_column_name = input_data.columns[-1]
    list_class = set(input_data[class_column_name])
    rata2 = {}
    std = {}
            
    for column in list_columns:
        for a_class in list_class:
            rata2[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].mean()
            std[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
    
    return (rata2, std)

In [12]:
data_latih.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
112,6.8,3.0,5.5,2.1,Virginica
128,6.4,2.8,5.6,2.1,Virginica
141,6.9,3.1,5.1,2.3,Virginica
98,5.1,2.5,3.0,1.1,Versicolor
53,5.5,2.3,4.0,1.3,Versicolor


Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [13]:
# test hitung_rata2_std_kelas()
test_rata2, test_std = hitung_rata2_std_kelas(data_latih)
print(test_rata2)
print(test_std)

{('Virginica', 'sepal.length'): 6.678378378378378, ('Versicolor', 'sepal.length'): 5.882051282051282, ('Setosa', 'sepal.length'): 4.997727272727273, ('Virginica', 'sepal.width'): 3.010810810810811, ('Versicolor', 'sepal.width'): 2.764102564102565, ('Setosa', 'sepal.width'): 3.4045454545454543, ('Virginica', 'petal.length'): 5.6324324324324335, ('Versicolor', 'petal.length'): 4.225641025641025, ('Setosa', 'petal.length'): 1.4636363636363634, ('Virginica', 'petal.width'): 2.0621621621621617, ('Versicolor', 'petal.width'): 1.3307692307692307, ('Setosa', 'petal.width'): 0.2386363636363636}
{('Virginica', 'sepal.length'): 0.6119799992806297, ('Versicolor', 'sepal.length'): 0.5290864991820659, ('Setosa', 'sepal.length'): 0.35795844657090964, ('Virginica', 'sepal.width'): 0.318663688779482, ('Versicolor', 'sepal.width'): 0.33520392810094735, ('Setosa', 'sepal.width'): 0.3728922442111284, ('Virginica', 'petal.length'): 0.5710876825332001, ('Versicolor', 'petal.length'): 0.5097563652930831, ('S

In [14]:
import math

def hitung_likelihood_gaussian(data, rata2, std):
    hasil = 1/(math.sqrt(2 * np.pi * np.power(std, 2))) * math.exp( (-np.power(data-rata2, 2))/ 2 * np.power(std, 2))
    return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-masing fitur dan kelas. 

In [15]:
def training_naive_bayes_gaussian(data_latih):

    class_column_name = data_latih.columns[-1]
    prior = hitung_prior(data)
    rata2, std = hitung_rata2_std_kelas(data_latih)
    list_class = set(data_latih[class_column_name])
    list_columns = data_latih.columns[:-1]
    
    model = {}

    model['prior'] = prior
    model['rata2'] = rata2
    model['std'] = std
    model['list_class'] = list_class
    model['list_columns'] = list_columns
    
    return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [16]:
model = training_naive_bayes_gaussian(data_latih)
print(model)

{'prior': Counter({'sepal.length': 0.006666666666666667, 'sepal.width': 0.006666666666666667, 'petal.length': 0.006666666666666667, 'petal.width': 0.006666666666666667, 'variety': 0.006666666666666667}), 'rata2': {('Virginica', 'sepal.length'): 6.678378378378378, ('Versicolor', 'sepal.length'): 5.882051282051282, ('Setosa', 'sepal.length'): 4.997727272727273, ('Virginica', 'sepal.width'): 3.010810810810811, ('Versicolor', 'sepal.width'): 2.764102564102565, ('Setosa', 'sepal.width'): 3.4045454545454543, ('Virginica', 'petal.length'): 5.6324324324324335, ('Versicolor', 'petal.length'): 4.225641025641025, ('Setosa', 'petal.length'): 1.4636363636363634, ('Virginica', 'petal.width'): 2.0621621621621617, ('Versicolor', 'petal.width'): 1.3307692307692307, ('Setosa', 'petal.width'): 0.2386363636363636}, 'std': {('Virginica', 'sepal.length'): 0.6119799992806297, ('Versicolor', 'sepal.length'): 0.5290864991820659, ('Setosa', 'sepal.length'): 0.35795844657090964, ('Virginica', 'sepal.width'): 0.3

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [17]:
def testing_naive_bayes_gaussian(model, data_uji):
    prior = model['prior']
    rata2 = model['rata2']
    std = model['std']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = dict.fromkeys(list_class, 1)

    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] = posterior[a_class] * hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
        posterior[a_class] = posterior[a_class] * prior[a_class]
    kelas_uji = max(posterior, key=posterior.get)
    return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [18]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model, data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Virginica
Virginica


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [19]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
    prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total==label_uji))

Total prediksi benar:  13


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [20]:
! wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

--2024-03-27 21:12:43--  https://dataset-ppm.s3.amazonaws.com/car_sample.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.219.142.68, 52.219.94.252, 52.219.108.220, ...
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.219.142.68|:443... 

connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-03-27 21:12:44 ERROR 403: Forbidden.



In [21]:
import pandas as pd
data = pd.read_csv('car_sample.csv')
data.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [30]:
tugas_latih, tugas_uji = train_test_split(data, train_size=0.7, stratify=data['class'])

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [31]:
tugas_label_uji = tugas_uji.pop('class')
tugas_label_uji.head()

218    unacc
266      acc
841    unacc
273    unacc
381    unacc
Name: class, dtype: object

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [32]:
prior_tugas = hitung_prior(tugas_latih['class'])
print("Prior: ", prior_tugas)

Prior:  Counter({'unacc': 0.7005789909015715, 'acc': 0.22249793217535152, 'good': 0.03970223325062035, 'vgood': 0.03722084367245657})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [75]:
def hitung_likelihood_multinomial(data_latih):
    if not isinstance(data_latih, pd.DataFrame):
        data_latih = pd.DataFrame(data_latih)
        
    features = data_latih.columns[:-1]
    target_col = data_latih.columns[-1]
    target_classes = pd.value_counts(data_latih[target_col])

    likelihood = {}
    for feature in features:
        # print("Now calculating for feature: ", feature)
        for feature_class in set(data_latih[feature]):
            # print(f"Feature {feature} feature_class {feature_class}")
            for target_class in target_classes.index:
                nc = target_classes[target_class]
                nc_w = ((data_latih[feature] == feature_class) & (data_latih[target_col] == target_class)).sum()
                likelihood[(feature, feature_class, target_class)] = nc_w/nc
                # print(f"Post. prob for {feature}, {feature_class}, {target_class} : ", likelihood[((feature, feature_class, target_class))])

    return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [76]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'med', 'unacc'): 0.23258559622195984, ('buying', 'med', 'acc'): 0.29739776951672864, ('buying', 'med', 'good'): 0.2708333333333333, ('buying', 'med', 'vgood'): 0.3333333333333333, ('buying', 'high', 'unacc'): 0.2644628099173554, ('buying', 'high', 'acc'): 0.26765799256505574, ('buying', 'high', 'good'): 0.0, ('buying', 'high', 'vgood'): 0.0, ('buying', 'low', 'unacc'): 0.21605667060212513, ('buying', 'low', 'acc'): 0.2379182156133829, ('buying', 'low', 'good'): 0.7291666666666666, ('buying', 'low', 'vgood'): 0.6666666666666666, ('buying', 'vhigh', 'unacc'): 0.28689492325855964, ('buying', 'vhigh', 'acc'): 0.1970260223048327, ('buying', 'vhigh', 'good'): 0.0, ('buying', 'vhigh', 'vgood'): 0.0, ('maint', 'med', 'unacc'): 0.22195985832349469, ('maint', 'med', 'acc'): 0.3271375464684015, ('maint', 'med', 'good'): 0.3333333333333333, ('maint', 'med', 'vgood'): 0.4666666666666667, ('maint', 'high', 'unacc'): 0.2597402597402597, ('maint', 'high', 'acc'): 0.27137546468401486, ('mai

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [102]:
def training_naive_bayes_multinomial(data_latih):
    model = {}

    model['prior'] = hitung_prior(data_latih.iloc[-1])
    model['likelihood'] = hitung_likelihood_multinomial(data_latih)
    model['classes'] = set(data_latih[data_latih.columns[-1]])
    model['features'] = data_latih.columns[:-1]

    return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [106]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

In [117]:
model_tugas

{'prior': Counter({'med': 0.2,
          'high': 0.2,
          'big': 0.2,
          'low': 0.2,
          'unacc': 0.2}),
 'likelihood': {('buying', 'med', 'unacc'): 0.23258559622195984,
  ('buying', 'med', 'acc'): 0.29739776951672864,
  ('buying', 'med', 'good'): 0.2708333333333333,
  ('buying', 'med', 'vgood'): 0.3333333333333333,
  ('buying', 'high', 'unacc'): 0.2644628099173554,
  ('buying', 'high', 'acc'): 0.26765799256505574,
  ('buying', 'high', 'good'): 0.0,
  ('buying', 'high', 'vgood'): 0.0,
  ('buying', 'low', 'unacc'): 0.21605667060212513,
  ('buying', 'low', 'acc'): 0.2379182156133829,
  ('buying', 'low', 'good'): 0.7291666666666666,
  ('buying', 'low', 'vgood'): 0.6666666666666666,
  ('buying', 'vhigh', 'unacc'): 0.28689492325855964,
  ('buying', 'vhigh', 'acc'): 0.1970260223048327,
  ('buying', 'vhigh', 'good'): 0.0,
  ('buying', 'vhigh', 'vgood'): 0.0,
  ('maint', 'med', 'unacc'): 0.22195985832349469,
  ('maint', 'med', 'acc'): 0.3271375464684015,
  ('maint', 'med', '

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [126]:
def testing_naive_bayes_multinomial(model, data_uji):
    class_prediction = "N/A"
    probability = {}

    for target_class in model['classes']:
        probability[target_class] = 1
    
    if (len(model['features']) == len(data_uji)):
        for feature in model['features']:
            # print("Calculating feature: ", feature)
            # print("Referenced value: ", data_uji[feature])
            for target_class in model['classes']:
                probability[target_class] *= model['likelihood'][(feature, data_uji[feature], target_class)]
                # print(f"Feature {feature} target class {target_class}: ", probability[target_class])

    highest_probability = max(probability, key=probability.get)

    return highest_probability
    

Lakukan pengujian menggunakan sebuah data uji

In [128]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

acc
unacc


Lakukan pengujian terhadap semua data uji.

In [129]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  352


In [130]:
len(tugas_label_uji)

519

### Penjelasan

In [132]:
print(f"Jumlah data train: {tugas_latih.shape[0]}")
print(f"Jumlah data test: {tugas_uji.shape[0]}")
print(f"Jumlah prediksi benar: {sum(prediksi_total==tugas_label_uji)}")
print(f"Akurasi model: {sum(prediksi_total==tugas_label_uji)/len(tugas_label_uji)}")

Jumlah data train: 1209
Jumlah data test: 519
Jumlah prediksi benar: 352
Akurasi model: 0.6782273603082851


Pada tugas praktikum kali ini, dibuat _functions_ untuk membuat model pembelajaran mesin untuk klasifikasi dataset "Car Evaluation" dengan algoritma Multinomial Naive-Bayes. 

Berikut langkah kerja yang dilakukan:
1. Splitting train dan test data dengan rasio 7:3
2. Perhitungan informasi _prior probability_ untuk setiap target class dengan memanggil fungsi hitung_prior(). Fungsi ini mengembalikan nilai probabilitas untuk masing-masing target class berdasarkan training data.
3. Perhitungan informasi _likelihood_ per fitur per target class dengan memanggil fungsi hitung_likelihood(). Fungsi ini akan melakukan iterasi untuk setiap feature, dan pada masing-masing feature akan dilakukan iterasi lagi pada setiap target class yang ada untuk melakukan perhitungan likelihood berdasarkan rumus Multinomial Naive-Bayes. 
4. Pembuatan model klasifikasi Multinomial Naive-Bayes dengan pemanggilan fungsi _wrapper_ training_naive_bayes_multinomial(). Fungsi ini akan mengembalikan model klasifikasi dalam bentuk dictionary yang berisikan informasi mengenai prior probability, likelihood, target classes, dan juga informasi mengenai fitur yang terdapat di dalam dataset.
5. Pembuatan fungsi untuk melakukan uji klasifikasi _single_ data point dengan menggunakan model yang telah dihasilkan pada langkah nomor 4. Proses testing ini dilakukan dengan memanggil fungsi testing_naive_bayes_multinomial(). Fungsi ini akan mengambil informasi nilai masing-masing feature dari _single_ data point dan menghitung posterior probability untuk masing-masing target class sesuai dengan informasi _likelihood_ pada model. Data point ini selanjutnya akan diklasifikasikan ke dalam kelas dengan nilai probabilitas tertinggi.

Pada langkah akhir, dilakukan proses pengujian model untuk semua data test yang berjumlah 519 data point. Dari pengujian di atas, didapatkan 352 data point memiliki kelas prediksi yang sesuai dengan kelas asli, menunjukkan akurasi model sebesar 68%.